# <span style="color:teal;">CIS 211 Project 8:  &nbsp; Bears and Fish</span>

##### Due 11:00 P.M. Thursday June 8

##### <span style="color:red">Group Members:</span>

For this project you are allowed to work in small groups of up to 3 people.

If you work in a group only one group member should upload this notebook to Canvas.  Edit this cell to include the name and DuckID for each member of the group:

**Name:**

**Name:**

**Name:**

##  <span style="color:teal;">Overview</span> 

The last project this term is a cellular automaton style biological simulation.  

The sytem is described in Chapter 11 of the Miller and Ranum text.  **Read the description of the simulation in the textbook before you start working on this project.**  Your code will be quite a bit different than the code in the book, but the rules of the simulation are the same, and you need to know how objects interact.

The project has three main parts:
* A class named World that defines a 2D grid where cells are accessed according to their row and column coordinates
* Classes for Bear and Fish, the organisms that can inhabit the world
* Two top level functions, `wbf` and `step_system`; the first will make a world and populate it at random with bears and fish, and the second will run a simulation using that world.

We're written `step_system` for you -- it's in the last code cell in this notebook -- but you need to design, implement, and test the classes and the `wbf` function.

##  <span style="color:teal;">Libraries</span> 

Your program will need to use `numpy` (a numeric processing library) and `random` (the builtin random number generator module).  You can import additional modules if you wish; add the import statements to this code cell.

In [136]:
IPython = (__doc__ is not None) and ('IPython' in __doc__)
Main    = __name__ == '__main__'

import numpy as np
import random

##  <span style="color:teal;">Event Log</span> 

A useful debugging technique is to save descriptions of events in a log.  Any code in the simulator, whether it is the top level function or a method in one of your classes, can call a function named `log`, passing it a description of an event that just occurred.  Examples might be 
```
new Fish in cell (x,y)
```
or 
```
Bear in cell (x,y) eats Fish in cell (x,y)
```

The function below uses a global variable named `logging`.  If you want to take advantage of the `log` function:
* edit the cell to set `logging` to True
* add calls to `log` at various points in your methods

Later, when doing large scale simulations, you can turn off logging simply by setting `logging` to False before you call the top level simulation function.

**Note:** &nbsp; you can change the definition of the `log` function however you want, _e.g._ you can have it write log messages to a file or print additional information.

In [137]:
logging = True

def log(message):
    if logging:
        print(message)

##  <span style="color:teal;">World (20 points)</span>

Write the code for the World class in the code cell below.

The constructor will be passed the grid size (number of rows and columns), and it should initialize all the cells to `None`.

Define the following methods:
* `add` will be passed an object and a location; it should store the object in the specified location 
* `fetch` should return the current contents of a location (which may be `None`)
* `remove` should set a location to `None`.

Finally, define a method named `biota` that will return a list of all items currently in the world.

You can have additional instance methods, class variables, or class methods.  Make sure you describe any new additions in the documentation.

##### <span style=color:red>Code</span> 

**Important** Write the definition of your World class in the following code cell. Do not delete or move this cell.

In [138]:
class World:
     
    def __init__(self, maxX, maxY):
        self.maxX = maxX
        self.maxY = maxY
        self.thingList = []
        self.grid = []
        
        self.grid = np.array([None] * maxX * maxY).reshape(self.maxX, self.maxY)
        
    def add(self, athing, coords):
        x = coords[0]
        y = coords[1]
        self.grid[x][y] = athing
        self.thingList.append(athing)
        
    def remove(self, athing, coords):
        x = coords[0]
        y = coords[1]
        if self.emptyLocation(x,y) == False:
            self.grid[x][y] == None
            self.thingList.remove(athing)
        
    def moveThing(self, oldx, oldy, newx, newy):
        self.grid[newy][newx] = self.grid[oldy][oldx]
        self.grid[oldy][oldx] = None
    
    def emptyLocation(self, x, y):
        if self.grid[y][x] == None:
            return True
        else:
            return False
        
    def fetch(self, coords):
        x = coords[0]
        y = coords[1]
        return self.grid[x][y]
    
    def biota(self):
        return self.thingList
    
    def __repr__(self):
        return str(self.grid)
   

##### <span style="color:red">Tests:</span>

Use the following code cell as a "sandbox" if you want to do your own tests.  You can add additional cells here if you want.

##### <span style="color:red">Autograder Tests:</span>

**Important:** &nbsp;  the code cells in this section will be used by `nbgrader` to run automated tests.  Do not move, delete or alter these cells in any way.

In [139]:
# A new world has no objects
w0 = World(5,5)

assert len(w0.biota()) == 0

In [140]:
# Test the add and fetch operators
w1 = World(5,5)

w1.add('hello', (0,0))
w1.add('world', (1,1))

assert w1.fetch((0,0)) == 'hello'
assert w1.fetch((1,1)) == 'world'

In [141]:
# Test the remove and biota methods
w2 = World(5,5)

w2.add('hello', (0,0))
w2.add('world', (1,1))

assert sorted(w2.biota()) == ['hello','world']

w2.remove('hello', (0,0))
w2.remove('world', (1,1))

assert w2.biota() == []

##### <span style=color:red>Documentation</span> 

**Important** Write your documentation in the following markdown cell.  Do not delete or move this cell.

The World class is the basis of the simulation game. This allow us to put all our animals in the "world" which the grid. With the methods add and remove, we are allowed to add and remove objects(in this case) different animals o3nto the grid. 

## <span style="color:teal;">Fish and Bears</span>

To run a simulation we need to add a random collection of animals to the world.  The two types of animals in this simulation are fish and bears, and you will write class definitions named Fish and Bear that implement the behaviors of the animals.

### Fish Class

During the simulation a Fish object needs to behave as follows:

(1) Fish are susceptible to overcrowding:  if there are fish in 2 or more neighboring cells the fish dies (it's removed from the simulation)

(2) A fish can reproduce if it has been alive for a certain number of time steps: a random neighboring cell is chosen, and if that cell is empty, a new fish is placed in that cell

(3) A fish can move to another cell:  it picks a random direction, and if the neighboring cell in that direction is unoccupied the fish moves there

The constructor for the Fish class will be passed a reference to a World object and a location, in the form of a tuple with a row number and column number (the object needs to know its location so it can look for other objects in neighboring cells).

The class should include the following methods:
* `live` implements rules 1 and 2 shown above
* `move` implements rule 3
* `location` returns the current grid location (row and column) of the object

Define a class variable named `breed_interval` to specify how many time steps a fish must be alive before it reproduces; the initial value for this variable is 12.


### Bear Class

During the simulation a Bear object needs to behave as follows:

(1) A bear looks for fish in each adjacent cell; if it finds one or more fish it eats one at random 

(2) If a bear has not eaten for certain number of time steps it dies (it's removed from the simulation)

(3) A bear can reproduce if it has been alive for a certain number of time steps: a random neighboring cell is chosen, and if that cell is empty, a new bear is placed in that cell

(4) A bear can move to another cell:  it picks a random direction, and if the neighboring cell in that direction is unoccupied the bear moves there

Define a class variable named `breed_interval` to specify how many time steps a bear must be alive before it reproduces; the initial value for this variable is 8.  Define another class variable named `survive_without_food` to be the number of time steps a bear can live before it dies from starvation; the initial value for this variable is 10.

The constructor will be passed a reference to a World object and a location, in the form of a tuple with a row number and column number (the object needs to know its location so it can look for other objects in neighboring cells).

The class should include the following methods:
* `live` implements rules 1, 2, and 3 shown above
* `move` implements rule 4
* `location` returns the current grid location (row and column) of the object

### Animal (Base Class)?

From the descriptions above it should be apparent that fish and bears have some things in common.

For **full credit** on the coding and documnetation portions of this project you should define a class named Animal and use it as the base class for your Fish and Bear classes.  Some things to think about as you design your classes:
* are there behaviors or operations that are common to both and that can be implemented just once in Animal?
* perhaps a behavior or operation can be defined with a default in the Animal class, and then overridden in the derived class?

You can still receive **partial credit** if you skip the Animal class and simply write completely separate Fish and Bear classes.  None of the unit tests assume there is a class named Animal.

### Details and Hints

**Incremental Development:**  One strategy you might consider is to write a complete implementation for one class, either Bear or Fish.  After you have debugged the class and it passes its unit tests you'll have a better idea of what to implement in the other class.  Then you can start moving common behaviors to the Animal class while you are writing the other derived class.

**Fish:** &nbsp; The way the simulation is defined a fish might be eaten before the top level simulation calls the `live` method.  Your `live` method should check to make sure the fish is still alive.  The easiest way to do this is to include an instance variable named `_alive` that is set to True when the fish is initialized and set to False when it dies.

You can have additional instance methods, class variables, or class methods.  Make sure you describe any new additions in the documentation.

**Animal:** &nbsp; Keep this in mind when you design the class hierarchy: every object has an attribute named ``__class__`` (with two underscores before and after the name).  It is a reference to the class an object was defined with.  As an example of how to use it, consider what would happen if we want to define `reproduce` in the Animal class so it is inherited by both Fish and Bear classes.  We need to know the value of `breed_interval` in each object's own class.  We can find this value using the expression
```
self.__class__.breed_interval
```
This will be a reference to `Fish.breed_interval` or `Bear.breed_interval`, depending on whether a `reproduce` was called with Fish object or Bear object.


### <span style="color:teal;">Animal (15 points)</span>

##### <span style=color:red>Code</span> 

If you implement the Animal class write the definition in the following code cell

**Important:** &nbsp; Do not delete or move this cell.

In [157]:
class Animal:
    
    def __init__(self,world, coords):
        
        self.xpos = coords[0]
        self.ypos = coords[1]
        self.world = world
        
        self.breedTick = 0
        self._alive = True
        self.world.add(self,coords) #everytime I create an animal it will automatically add it to the grid & thingList
        self.time_alive = 0
        
    def setX(self,newx):
        self.xpos = newx
        
    def setY(self,newy):
        self.ypos = newy
        
    def getX(self):
        return self.xpos
    
    def getY(self):
        return self.ypos

    def aroundAnimal(self):
        '''
        Returns a list of things around this thing
        '''
        list_thing = []
        y = self.ypos
        x = self.xpos
        for item in [((y -1) % self.world.maxX), (y % self.world.maxY), ((y + 1) % self.world.maxY)]:
            for thing in [((x-1) % self.world.maxX),(x % self.world.maxX), ((x+1) % self.world.maxX)]:
                list_thing.append((thing,item))
            
                
        list_thing.remove((x,y))
        return list_thing
    
    def reproduce(self):            
        if (self._alive and (self.time_alive % self.__class__.breed_interval == 0)):
            none_list = []
            for item in self.aroundAnimal():
                if self.world.fetch(item) == None:
                    none_list.append(item)
            add_animal = none_list[random.randint(0,len(none_list)-1)]
            select_animal = self.__class__(self.world,add_animal)
        
    def location(self):
        return (self.xpos,self.ypos)
    
    def move(self):
        move_list = []
        for item in self.aroundAnimal():
            if self.world.fetch(item) == None:
                move_list.append(item)  
        addAnimal = move_list[random.randint(0,len(move_list)-1)]
        self.xpos = addAnimal[0]
        self.ypos = addAnimal[1]
        self.world.moveThing(self.xpos,self.ypos,addAnimal[0],addAnimal[1])
    

        
        
        
        

##### <span style="color:red">Tests:</span>

There are **no unit tests for the Animal class.**  If you implement this class it will be tested automatically as part of the unit tests for Bear and Fish.

You can still make your own tests of the base class if you wish.  Add expressions that create and test Animal objects to the following "sandbox" cells.

##### <span style=color:red>Documentation</span> 

If you implement the Animal class write your documentation in the following markdown cell.  

**Important** Do not delete or move this cell.

This class Animal allows all animals(bears and fish) in this simulation to do basic living actions that any animal can do like reproduce and move as well as find the location of the animal. This class uses methods that will be used for both animals.

### <span style="color:teal;">Fish (20 points)</span>

##### <span style=color:red>Code</span> 

**Important:** &nbsp; Write the Python code for your Fish class in the following code cell.  Do not delete or move this cell.

In [158]:
class Fish(Animal):
    
    breed_interval = 12

    def check(self):
        count = 0
        for item in self.aroundAnimal():
            if type(self.world.fetch(item)) == Fish:
                count += 1
        if count >= 2:
            self.world.remove(self,self.location())
            self._alive = False
                
        
    def __repr__(self):
        return u"\U0001F41F"
    
    def setWorld(self,aworld):
        self.world = aworld
        
        
    def live(self):
        self.check()
        self.reproduce()
        
    

##### <span style="color:red">Tests:</span>

Use the following code cell as a "sandbox" if you want to do your own tests.  You can add additional cells here if you want.

##### <span style="color:red">Autograder Tests:</span>

**Important:** &nbsp;  the code cells in this section will be used by `nbgrader` to run automated tests.  Do not move, delete or alter these cells in any way.

In [159]:
# A new world has no objects
fw1 = World(5,5)
assert len(fw1.biota()) == 0

# After adding a fish there should be one object
f1 = Fish(fw1, (2,2))
assert len(fw1.biota()) == 1

# Test the location method
assert f1.location() == (2,2)

In [160]:
# Setting breed_interval to 1 should cause a fish to reproduce when live is called
fw2 = World(5,5)
Fish.breed_interval = 1
f2 = Fish(fw2, (2,2))
f2.live()
assert len(fw2.biota()) == 2

# Reset the interval to original value for remaining tests
Fish.breed_interval = 12

In [161]:
# Make three fish, the one in the middle should die from overcrowding
fw3 = World(5,5)
f3 = Fish(fw3, (2,2))
Fish(fw3, (1,1))
Fish(fw3, (3,3))
f3.live()
assert len(fw3.biota()) == 2

In [162]:
# When a fish moves it should be within one cell of its original location
fw4 = World(5,5)
f4 = Fish(fw4, (2,2))
f4.move()
r, c = f4.location()
assert (r,c) != (2,2)
assert abs(r-2) <= 1 and abs(c-2) <= 1

##### <span style=color:red>Documentation</span> 

**Important:** &nbsp; Write the documentation for your Fish class in the following markdown cell.  Do not delete or move this cell.

This classes uses inheritance to become a sublcass of the Animal class. Fish and Bears in this simulation had a bunch of the same functions but they each have little differences. The fish get eaten by the bears which cause them to die more often.

###  <span style="color:teal;">Bear (20 points)</span>

##### <span style=color:red>Code</span> 

**Important:** &nbsp; Write the Python code for your Bear class in the following code cell.  Do not delete or move this cell.

In [163]:
class Bear(Animal):
    
    breed_interval = 8
    survive_without_food = 10
    
    def __init__(self, world, coords):
        
        Animal.__init__(self,world,coords)
        
        self.starveTick = 0
        
        
    def __repr__(self):
        return u"\U0001F43B"
        
    def liveALittle(self):
        self.breedTick = self.breedTick + 1
        if self.breedTick >= 8:
            self.tryToBreed()
            
        self.tryToEat()
        
        if self.starveTick == 10:
            self.world.delThing(self)
        else:
            self.tryToMove()
            
    def tryToEat(self):
        offsetList = [(-1,1) ,(0,1) ,(1,1),(-1,0), (1,0),(-1,-1), (0,-1), (1,-1)]
        
        adjprey = []
        for offset in offsetList:
            newx = self.xpos + offset[0]
            newy = self.ypos + offset[1]
            if 0 <= newx < self.world.getMaxX() and 0 <= newy < self.world.getmaxY():
                if(not seld.world.emptyLocation(newx,newy)) and isinstance(self.world.lookAtLocation(newx, newy),Fish):
                    adjprey.append(self.world.lookAtLocation(newx,newy))
                    
        if len(adjprey) > 0:
            randomprey = adjprey[random.randrange(len(adjprey))]
            preyx = randomprey.getX()
            preyy = randomprey.getY()
            
            self.world.delThing(randomprey)
            self.move(preyx,preyy)
            self.starveTick = 0
        else:
            self.starveTick = self.starveTick + 1

        '''live implements rules 1, 2, and 3 shown above
        move implements rule 4
        location returns the current grid location (row and column) of the object'''

    def survive_without_food_function(self):
        plenty_of_fish = []
        for item in self.aroundAnimal():
            if type(self.world.fetch((item[0],item[1]))) == Fish:
                plenty_of_fish.append(item)
        if len(plenty_of_fish) > 0:
            fish_murder = plenty_of_fish[random.randint(0,len(plenty_of_fish)-1)]
            self.world.remove(self,fish_murder)
            self._alive = False
        else:
            self.starveTick += 1
        if self.starveTick == Bear.survive_without_food:
            self.world.remove(self,(self.xpos,self.ypos))
            self._alive = False
                
    def live(self):
        self.survive_without_food_function()
        self.reproduce()

##### <span style="color:red">Tests:</span>

Use the following code cell as a "sandbox" if you want to do your own tests.  You can add additional cells here if you want.

##### <span style="color:red">Autograder Tests:</span>

**Important:** &nbsp;  the code cells in this section will be used by `nbgrader` to run automated tests.  Do not move, delete or alter these cells in any way.

In [164]:
# Test the Bear constructor and location method
bw1 = World(5,5)
b1 = Bear(bw1, (1,1))
assert len(bw1.biota()) == 1
assert b1.location() == (1,1)

In [165]:
# Repeat the reproduction test for Bears
bw2 = World(5,5)
Bear.breed_interval = 1
b2 = Bear(bw2, (2,2))
b2.live()
assert len(bw2.biota()) == 2
Bear.breed_interval = 8

In [166]:
# Make fish for the bear to eat, count the number of objects after eating
bw3 = World(5,5)
b3 = Bear(bw3, (2,2))
Fish(bw3, (1,1))
Fish(bw3, (3,3))
b3.live()
assert len(bw3.biota()) == 2

In [167]:
# Setting the survival limit to 1 should cause a Bear to starve 
bw4 = World(5,5)
Bear.survive_without_food = 1
b4 = Bear(bw4, (2,2))
b4.live()
assert len(bw4.biota()) == 0
Bear.survive_without_food = 10

In [168]:
# Repeat the move test for bears
bw5 = World(5,5)
b5 = Bear(bw5, (2,2))
b5.move()
r, c = b5.location()
assert (r,c) != (2,2)
assert abs(r-2) <= 1 and abs(c-2) <= 1

##### <span style=color:red>Documentation</span> 

**Important:** &nbsp; Write the documentation for your Bear class in the following markdown cell.  Do not delete or move this cell.

The Bear is a subclass of the Animal class and it uses inheritance as well just like the FIsh class, but like the Fish class, the Bear class has some differences that make it unique from the the Fish. Unlike the Fish, the Bears in the simulation eat, which is shown in the function TryToEat, as well as them being able to survive_without_food, which is also a function in this class. 

##  <span style="color:teal;">The `wbf` Function (15 points)</span>

Fill in the body of the `wbf` function so it returns a new World object with the specified number of rows and columns and with the specified number of Bear and Fish objects at random locations.

When we grade your project we will call `wbf` to make a World object and then use the main loop (implemented by `step_system`) to run the simulation.

In [169]:
def wbf(nrows, ncols, nbears, nfish):
    '''
        This function uses a main loop which is implemented by step_system to help run the simulation.
    '''
    new_world = World(nrows,ncols)
    bear_number = 0
    fish_number = 0
    while bear_number < nbears:
        rows = random.randint(0,nrows-1)
        cols = random.randint(0,ncols-1)
        if new_world.fetch((rows,cols)) == None:
            bear_number += 1
            new_bear = Bear(new_world,(rows,cols))
    while fish_number < nfish:
        rows = random.randint(0,nrows-1)
        cols = random.randint(0,ncols-1)
        if new_world.fetch((rows,cols)) == None:
            fish_number += 1
            new_fish = Fish(new_world,(rows,cols))
    return new_world

##### <span style="color:red">Tests:</span>

Use the following code cell as a "sandbox" if you want to do your own tests.  You can add additional cells here if you want.

##### <span style="color:red">Autograder Tests:</span>

**Important:** &nbsp;  the code cells in this section will be used by `nbgrader` to run automated tests.  Do not move, delete or alter these cells in any way.

In [170]:
w = wbf(10,10,3,12)

dct = { Bear: 0, Fish: 0 }
for x in w.biota():
    dct[x.__class__] += 1

assert dct[Bear] == 3
assert dct[Fish] == 12

##### <span style=color:red>Documentation</span> 

**Important:** &nbsp; Write the documentation for your `wbf` function in the following markdown cell.  Do not delete or move this cell.

This function helps run the simulation by using the World class, Animal class, and the Bear class to run the simulation. It also imports random to help always give us different outcomes of the simulation. 

##  <span style="color:teal;">The `step_system` Function</span>

We've written this function for you.  It will run a single time step of the simulation.  Pass it a World object containing a grid populated with Bear and Fish objects and it will (a) see which animals survive, then (b) allow all the animals to move to a new location.

In [171]:
def step_system(world):
    for x in world.biota():
        x.live()
    for x in world.biota():
        x.move()

The code cell below is an example of how to run a complete simulation.  Call `wbf` to make a World populated by Bear and Fish objects, and then run the simulation for some number of iterations.

You can run this cell to do an "integration test" of your code.  You can also copy it and modify it for the exercises in the next section.

In [172]:
if IPython:
    logging = True
    w = wbf(10,10,3,12)
    for i in range(3):
        print(w)
        step_system(w)

[[None None None None None 🐻 None None None 🐟]
 [None None None None None None None None None 🐟]
 [None None None 🐻 None None None None None None]
 [None None None None 🐟 None None None None None]
 [None None None None None None 🐟 None None 🐟]
 [None None None 🐟 None None None None 🐟 None]
 [None None None None None None None None None None]
 [🐻 None 🐟 None None None 🐟 None None None]
 [🐟 None None None None None 🐟 None None None]
 [None None None None None None None None 🐟 None]]
[[None None 🐻 🐻 None 🐻 None None None None]
 [None None None 🐻 None None None None 🐟 🐟]
 [None None None 🐻 None None None None None None]
 [None None None None 🐟 🐟 None None None None]
 [None None None None None 🐟 🐟 None None None]
 [🐟 🐟 None 🐟 None None None None 🐟 None]
 [None None None 🐟 None None 🐟 None 🐟 None]
 [🐻 None 🐟 None None None 🐟 🐟 None None]
 [None 🐟 🐻 None None None 🐟 None None None]
 [None None None 🐻 None None None 🐟 🐟 None]]
[[🐟 None None 🐻 None 🐻 None None None None]
 [None 🐟 🐟 None None No

##  <span style="color:teal;">Experiments (10 points)</span>

Run some experiments with the top level simulation loop and describe the results in the markdown cell below.  Some ideas of things to try:
* The settings for the Bear class `breed_interval` and `survive_without_food` variables come from the textbook.  Will the world ever run out of bears with these settings?
* Change the settings so `Bear.breed_interval` is larger than `Bear.survive_without_food`.  How does that change the outcome?
* Set the `breed_interval` counter for the Fish class to a smaller number (e.g. 4) so the world has more fish.  What effect does that have?
* Write a function that runs the simulation for a specified number of generations, or until there are no more objects left in the grid.  What combination of parameters leads to the largest number of time steps before the simulation halts?

When you change diffrent methods throughout the classses in Bear and Fish you can notice the change in different outcomes. When you change the breed intervals you notice the change in the number of fish and bears. Sometimes there will be more bears and less fish, or vice versa. It is intersting to run and see how the simulation really gives you a feel for changes in the environment based of the changes you make to the ecosystem in which they live in.

##  <span style="color:teal;">Extra Credit Ideas</span>

Here are some suggestions for ways to extend the simulation.  We will consider other types of extensions -- send a request to `conery@uoregon.edu` with your proposal.

* Implement the Plant class described in Section 10.7 of the textbook, and modify the Fish class so fish eat plants and die if they don't find enough food.

* If you implement Plant, how does that affect the class hierarchy?  Is there an even more general class called Organism, with Plants and Animals subtypes of organism?

* Experiment with data structures: make a second version of the World class, but use a list-of-lists approach to making the grid.  Which is more efficient, a list of lists or a numpy array?  Which scales better when larger worlds are used in the simulation?

* Implement a GUI using `tkinter` that is similar to the Solar System GUI.  A canvas should display the world along with images for the Bear and Fish objects (you can download `Bear.gif` and `Fish.gif` from Canvas).  Use a spinbox or text entry box to specify the number of Bear and Fish objects and the number of time steps to run.  Include a Run button to start the simulation.
